In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\jjtre\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
!pip install joblib

In [3]:
import pandas as pd
import os
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [5]:
training_drop_na_df = pd.read_csv(r"C:\Users\jjtre\Documents\UCSD Coding Bootcamp Data Science\Final Project\testing_cleaned_tweets_stopword_filter_dropnan.csv", encoding='utf-8')
training_drop_na_df.head()

,tweet_polarity,tweet_text,clean_tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50 rest go ...
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see


In [6]:
training_drop_na_df['tweet_polarity'].value_counts()

4    796299
0    796229
Name: tweet_polarity, dtype: int64

In [7]:
# set the label for the number of positive and negative tweets
label = training_drop_na_df['tweet_polarity'].to_list()
labels = label[0: 5000] + label[796229: 801229]
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [8]:
labels_df = pd.DataFrame(labels)
labels_df["labels"] = pd.DataFrame(labels)
labels_df.drop(labels_df.columns[[0]], axis=1, inplace=True)
labels_df.head()

,labels
0,0
1,0
2,0
3,0
4,0


In [9]:
y = labels_df["labels"].values.reshape(-1, 1)
print(y)
print(y.shape)

[[0]
 [0]
 [0]
 ...
 [4]
 [4]
 [4]]
(10000, 1)


In [10]:
# grab the percentage of the neg(0) and pos(4) cleaned tweets
feature = training_drop_na_df['clean_tweets'].to_list()
features = feature[0: 5000] + feature[796229: 801229]
features

['awww bummer shoulda got david carr third day',
 'upset update facebook texting might cry result school today also blah',
 'dived many times ball managed save 50 rest go bounds',
 'whole body feels itchy like fire',
 'behaving mad see',
 'whole crew',
 'need hug',
 'hey long time see yes rains bit bit lol fine thanks',
 'nope',
 'que muera',
 'spring break plain city snowing',
 'pierced ears',
 'bear watch thought ua loss embarrassing',
 'counts idk either never talk anymore',
 'would first gun really though zac snyder doucheclown',
 'wish got watch miss premiere',
 'hollis death scene hurt severely watch film wry directors cut',
 'file taxes',
 'ahh ive always wanted see rent love soundtrack',
 'oh dear drinking forgotten table drinks',
 'day get much done',
 'one friend called asked meet mid valley today time sigh',
 'baked cake ated',
 'week going hoped',
 'blagh class 8 tomorrow',
 'hate call wake people',
 'going cry sleep watching marley',
 'im sad miss lilly',
 'ooooh lol lesli

In [11]:
# use TF_IDF vectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
features = vectorizer.fit_transform(features).toarray()

In [12]:
from sklearn.model_selection import train_test_split
X = features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
print(y_train)
print(y_train.shape)

[[4]
 [4]
 [4]
 ...
 [4]
 [0]
 [0]]
(7500, 1)


In [14]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train_scaler = X_scaler.transform(X_train)
X_test_scaler = X_scaler.transform(X_test)
y_train_scaler = y_scaler.transform(y_train)
y_test_scaler = y_scaler.transform(y_test)

C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaler, y_train_scaler)

C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [16]:
print('Train Acc: %.5f' % model.score(X_train_scaler, y_train_scaler))

Train Acc: 0.83387


In [19]:
# Model Accuracy
print('Test Acc: %.5f' % model.score(X_test_scaler, y_test_scaler))

Test Acc: 0.68920


In [17]:
target_names = ["Negative", "Positive"]

In [18]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test_scaler)
print(classification_report(y_test_scaler, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.69      0.69      0.69      1250
    Positive       0.69      0.69      0.69      1250

   micro avg       0.69      0.69      0.69      2500
   macro avg       0.69      0.69      0.69      2500
weighted avg       0.69      0.69      0.69      2500



In [20]:
# try a random search
from sklearn.model_selection import RandomizedSearchCV

In [21]:
model.get_params()

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto_deprecated',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [22]:
C = [1.0, 2.0, 5.0, 10.0, 25.0, 50.0]
kernel = ["linear", "poly", "rbf", "sigmoid"]
degree = [1, 2, 3, 4, 5]
gamma = [0.0001, 0.001, 0.01, "scale", "auto"]
shrinking = [True, False]
probability= [True, False]
tol = [0.0001, 0.001, 0.01, 0.1]
param_grid = dict(C=C, kernel=kernel, degree=degree, gamma=gamma, shrinking=shrinking, probability=probability, tol=tol)

In [ ]:
import time
random = RandomizedSearchCV(model, param_distributions=param_grid, cv = 5, verbose=2, random_state=42, n_jobs=-1, iid=False)

start_time = time.time()
random_result = random.fit(X_train_scaler, y_train_scaler)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' sec')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
print(random.best_params_)
print(random.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = random.predict(X_test_scaler)
predictions

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_scaler, predictions,
                            target_names=["Negative", "Positive"]))

In [ ]:
import joblib
filename = 'twitter_SVM.sav'
joblib.dump(model, filename)